# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [0]:
import pandas as pd
import numpy as np
import folium


In [2]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: True
Cloning into 'DataScienceSS20'...
remote: Enumerating objects: 550, done.
remote: Total 550 (delta 0), reused 0 (delta 0), pack-reused 550
Receiving objects: 100% (550/550), 122.05 MiB | 12.62 MiB/s, done.
Resolving deltas: 100% (221/221), done.
Checking out files: 100% (210/210), done.


In [0]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv(path+'/DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [4]:
# install 
!pip install scikit-optimize

     |████████████████████████████████| 81kB 2.2MB/s 


### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [0]:
mask = X['pickup_year']< 2015
X_train = X[mask] 
y_train = y[mask]

X_test = X[mask==False] 
y_test = y[mask==False]

In [0]:
from skopt import BayesSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

pipe = Pipeline([('model', LinearRegression())])

clf = BayesSearchCV(
     pipe,
     {
          'model': 
          [
            LinearRegression(),
            SVR(kernel='rbf',C=10, gamma=0.0001),
            RandomForestRegressor(random_state=0, n_jobs=4, n_estimators=99, max_depth=23)
          ]
     },
     n_iter=32,
     n_jobs = 4,
     scoring='neg_mean_squared_error'
 )

In [37]:
clf.fit(X_train[:10000], y_train[:10000])

RuntimeError: ignored

In [38]:
clf.cv_results_

defaultdict(list,
            {'mean_fit_time': [14.705779790878296,
              13.996438455581664,
              16.57693428993225,
              13.983858585357666,
              16.65515990257263,
              13.876861095428467,
              16.54414529800415,
              16.586374378204347,
              0.016764020919799803,
              16.60833740234375],
             'mean_score_time': [2.1896780490875245,
              2.245754528045654,
              0.16235089302062988,
              2.3081761837005614,
              0.142340087890625,
              2.2293888092041017,
              0.17940421104431153,
              0.16782126426696778,
              0.002165555953979492,
              0.14847445487976074],
             'mean_test_score': [-34.48438355799728,
              -34.48438355799728,
              -17.860200116357397,
              -34.48438355799728,
              -17.8602001163574,
              -34.48438355799728,
              -17.8602001163574,
      

In [39]:
clf.best_params_

OrderedDict([('model',
              RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                                    max_depth=23, max_features='auto', max_leaf_nodes=None,
                                    max_samples=None, min_impurity_decrease=0.0,
                                    min_impurity_split=None, min_samples_leaf=1,
                                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                                    n_estimators=99, n_jobs=4, oob_score=False,
                                    random_state=0, verbose=0, warm_start=False))])